In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

from datasets import Dataset, DatasetDict, concatenate_datasets, load_dataset

/home/hosman/anaconda3/envs/torchEnv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-15 08:47:15.622283: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-15 08:47:15.631037: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739598435.640789   17186 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739598435.643902   17186 cuda_blas.cc

In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [3]:
# 1. Model ve Tokenizer'ı Yükle
model_name = "HuggingFaceTB/SmolLM2-360M-Instruct"  # Mistral 7B modeli
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [4]:
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})


In [ ]:
# 2. Tapaco Veri Setini Yükleyin
dataset = load_dataset("turkish-nlp-suite/InstrucTurca", split="train", cache_dir="/media/hosman/Yedek/Datasets/")  # Tapaco veri setini yükle
dataset2 = load_dataset("Metin/WikiRAG-TR", split="train", cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"question": "Input", "answer": "Output"})
dataset3 = load_dataset("kayrab/patient-doctor-qa-tr-167732",  cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"question_content": "Input", "question_answer": "Output"})

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

from datasets import Dataset, DatasetDict, concatenate_datasets, load_dataset

/home/hosman/anaconda3/envs/torchEnv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-15 08:44:35.894267: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-15 08:44:35.903420: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739598275.913150   16163 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739598275.916034   16163 cuda_blas.cc

In [ ]:
# 1. Model ve Tokenizer'ı Yükle
model_name = "HuggingFaceTB/SmolLM2-360M-Instruct"  # Mistral 7B modeli
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [ ]:
dataset,dataset2,dataset3 

(Dataset({
     features: ['Input', 'Output'],
     num_rows: 2575505
 }),
 Dataset({
     features: ['id', 'Input', 'Output', 'context', 'is_negative_response', 'number_of_articles', 'ctx_split_points', 'correct_intro_idx'],
     num_rows: 5999
 }),
 DatasetDict({
     train: Dataset({
         features: ['doctor_title', 'doctor_speciality', 'Input', 'Output'],
         num_rows: 503196
     })
     test: Dataset({
         features: ['doctor_title', 'doctor_speciality', 'Input', 'Output'],
         num_rows: 60000
     })
 }))

In [ ]:
dataset3 = concatenate_datasets([dataset3["train"], dataset3["test"]])

In [ ]:
dataset = concatenate_datasets([dataset, dataset2, dataset3])

In [ ]:
dataset

Dataset({
    features: ['Input', 'Output', 'id', 'context', 'is_negative_response', 'number_of_articles', 'ctx_split_points', 'correct_intro_idx', 'doctor_title', 'doctor_speciality'],
    num_rows: 3144700
})

In [ ]:
from datasets import Dataset
import numpy as np

# None içeren satırları temizleyen fonksiyon
def remove_none_rows(example):
    return example["Input"] is not None and example["Output"] is not None

# None değerleri içeren satırları filtrele
dataset = dataset.filter(remove_none_rows)

Filter: 100%|██████████| 3144700/3144700 [00:15<00:00, 196808.26 examples/s]


In [ ]:
from datasets import Dataset
import numpy as np

# Token uzunluklarını hesaplayan fonksiyon
def get_token_lengths(example):
    input_length = len(example["Input"])
    output_length = len(example["Output"])
    return {"input_length": input_length, "output_length": output_length}

# Tüm veri seti için hesaplama
token_lengths = dataset.map(get_token_lengths, batched=False)

# Maksimum ve ortalama token sayıları
max_input_length = max(token_lengths["input_length"])
max_output_length = max(token_lengths["output_length"])

avg_input_length = np.mean(token_lengths["input_length"])
avg_output_length = np.mean(token_lengths["output_length"])

# Sonuçları yazdır
print(f"📌 Maksimum Input Token Sayısı: {max_input_length}")
print(f"📌 Ortalama Input Token Sayısı: {avg_input_length:.2f}")
print(f"📌 Maksimum Output Token Sayısı: {max_output_length}")
print(f"📌 Ortalama Output Token Sayısı: {avg_output_length:.2f}")


Map:  53%|█████▎    | 1653090/3144600 [01:49<01:26, 17229.39 examples/s]

In [ ]:
# Filtreleme fonksiyonu
def filter_long_samples(example):
    input_length = len(example["Input"])
    output_length = len(example["Output"])
    
    # Eğer input ve output ortalamadan büyükse filtrele (False döndür)
    return not (input_length > 512 and output_length > 512)

# Yeni filtrelenmiş dataset
dataset = dataset.filter(filter_long_samples)

# Filtrelenmiş veri kümesi hakkında bilgi
print(f"✅ Filtrelenmiş veri seti satır sayısı: {len(dataset)}")


Filter: 100%|██████████| 2575505/2575505 [00:25<00:00, 102151.14 examples/s]

✅ Filtrelenmiş veri seti satır sayısı: 2340034


In [ ]:
from datasets import DatasetDict

# 1. Veriyi train ve test olarak ayırma
# Örneğin, dataset zaten tek bir büyük veri seti (örneğin "data") içeriyor
# Bunu %80 train ve %20 test olarak bölelim
train_dataset, temp_dataset = dataset.train_test_split(test_size=0.2, seed=42).values()

# 2. Test setini de %50 validation ve %50 test olarak bölelim
val_dataset, test_dataset = temp_dataset.train_test_split(test_size=0.5, seed=42).values()

# 3. Veriyi tokenizasyon işlemi için tokenize edelim
def tokenize_function(example):
    # Input ve Output'u tokenize et
    input_tokens = tokenizer(example["Input"], padding="max_length", truncation=True, max_length=512)
    output_tokens = tokenizer(example["Output"], padding="max_length", truncation=True, max_length=512)

    # Tokenized Input ve Output'u döndür
    return {
        "input_ids": input_tokens["input_ids"],
        "attention_mask": input_tokens["attention_mask"],
        "labels": output_tokens["input_ids"]
    }

# 4. Tokenize işlemini her bir split için uygulayalım
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True).remove_columns(["Input","Output"])
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True).remove_columns(["Input","Output"])
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True).remove_columns(["Input","Output"])

# Tokenize edilmiş veri setlerini birleştirebilirsiniz (opsiyonel)
final_dataset = DatasetDict({
    'train': tokenized_train_dataset,
    'validation': tokenized_val_dataset,
    'test': tokenized_test_dataset
})



Map:  47%|████▋     | 874000/1872027 [08:05<09:01, 1843.35 examples/s]

In [ ]:
tokenized_test_dataset[0]["input_ids"], tokenized_test_dataset[0]["attention_mask"]

([67,
  270,
  101,
  310,
  7466,
  106,
  5549,
  131,
  94,
  7170,
  662,
  360,
  20335,
  268,
  20335,
  43702,
  20335,
  41108,
  42,
  1065,
  20335,
  32647,
  100,
  20335,
  98,
  2432,
  28,
  14401,
  419,
  419,
  5549,
  131,
  1362,
  301,
  264,
  5549,
  131,
  1362,
  12771,
  89,
  536,
  43702,
  490,
  265,
  3345,
  512,
  9922,
  48676,
  501,
  369,
  336,
  953,
  508,
  1482,
  7560,
  43702,
  462,
  41934,
  280,
  2303,
  1558,
  22817,
  337,
  30,
  340,
  3345,
  369,
  20335,
  94,
  20335,
  106,
  577,
  305,
  314,
  85,
  476,
  6248,
  43702,
  490,
  1002,
  253,
  538,
  89,
  10225,
  1018,
  476,
  73,
  276,
  92,
  20335,
  32647,
  18,
  329,
  1437,
  20335,
  94,
  30,
  25442,
  101,
  42,
  11841,
  30,
  824,
  6737,
  81,
  30019,
  11512,
  501,
  1274,
  276,
  20335,
  83,
  20335,
  891,
  47,
  45788,
  1362,
  89,
  42,
  216,
  34,
  32,
  32,
  37,
  329,
  20335,
  92,
  20335,
  41108,
  24036,
  20335,
  100,
  20335,
  1

In [ ]:
tokenized_test_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 252488
})

In [ ]:
#model.gradient_checkpointing_enable()
#model.use_cache = False


In [ ]:
import math
import numpy as np
import wandb
import evaluate
from sentence_transformers import SentenceTransformer, util
from collections import Counter

# Metrikleri yükle
bleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")
chrf = evaluate.load("chrf")
bert_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Distinct-N hesaplayan fonksiyon
def compute_distinct_n(preds, n=2):
    all_ngrams = [tuple(preds[i:i+n]) for i in range(len(preds)-n+1)]
    return len(set(all_ngrams)) / max(1, len(all_ngrams))

# Metrikleri hesaplayan ana fonksiyon
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    results = {}

    try:
        loss = np.mean(preds)
        results["perplexity"] = math.exp(loss)
    except Exception as e:
        results["perplexity"] = f"error: {str(e)}"

    try:
        results["bleu"] = bleu.compute(predictions=preds, references=labels)["score"]
    except Exception as e:
        results["bleu"] = f"error: {str(e)}"

    try:
        rouge_scores = rouge.compute(predictions=preds, references=labels)
        results.update({
            "rouge-1": rouge_scores["rouge1"].mid.fmeasure,
            "rouge-2": rouge_scores["rouge2"].mid.fmeasure,
            "rouge-L": rouge_scores["rougeL"].mid.fmeasure,
        })
    except Exception as e:
        results["rouge-1"] = results["rouge-2"] = results["rouge-L"] = f"error: {str(e)}"

    try:
        results["chrf"] = chrf.compute(predictions=preds, references=labels)["score"]
    except Exception as e:
        results["chrf"] = f"error: {str(e)}"

    try:
        results["distinct-1"] = compute_distinct_n(preds, n=1)
        results["distinct-2"] = compute_distinct_n(preds, n=2)
    except Exception as e:
        results["distinct-1"] = results["distinct-2"] = f"error: {str(e)}"

    try:
        similarities = [util.pytorch_cos_sim(bert_model.encode(p), bert_model.encode(l)).item() for p, l in zip(preds, labels)]
        results["semantic_similarity"] = sum(similarities) / len(similarities)
    except Exception as e:
        results["semantic_similarity"] = f"error: {str(e)}"

    # WandB loglama
    wandb.log(results)
    return results


In [ ]:
import wandb
from transformers import TrainerCallback
import torch

class WandBQuestionCallback(TrainerCallback):
    def __init__(self, tokenizer, model, questions, device="cuda", log_interval=1000):
        self.tokenizer = tokenizer
        self.model = model.to(device)
        self.device = device
        self.questions = questions  # List of question strings
        self.log_interval = log_interval
        
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % self.log_interval == 0:
            wandb.log({"step": state.global_step})
            self.log_model_responses()

    def log_model_responses(self):
        responses = {}
        for question in self.questions:
            messages = [
                {"role": "system", "content": "Sen yardımsever bir asistansın"},
                {"role": "user", "content": question}
            ]
            input_text = self.tokenizer.apply_chat_template(messages, tokenize=False)
            inputs = self.tokenizer.encode(input_text, return_tensors="pt").to(self.device)
            outputs = self.model.generate(inputs, max_new_tokens=512, temperature=0.2, top_p=0.9, do_sample=True)
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            responses[question] = response
        
        wandb.log({"model_responses": responses})

In [ ]:
# Örnek sorular
questions = [
    "433 * b - 7420490 = -7413995 denklemini çöz.",
    "Türkiye'nin başkenti neresidir?",
    "E=mc^2 denkleminin fiziksel anlamı nedir?",
    "Merhaba.Nasılsın?"
]

# Callback'i oluştur
wandb_callback = WandBQuestionCallback(tokenizer, model, questions)

In [ ]:
# 4. Eğitim Ayarlarını Tanımlayın
training_args = TrainingArguments(
    output_dir="./SmolLM2-360M-Instruct-v1",  # Çıktı dizini
    evaluation_strategy="epoch",         # Değerlendirme adımları
    save_strategy="steps",               # Kaydetme adımları
    save_steps=300,                      # Her 500 adımda modeli kaydet
    logging_dir="./logs",                # Log dosyaları dizini
    logging_steps=500,                   # Her 100 adımda log yazdır
    learning_rate=2e-5,                  # Öğrenme oranı
    num_train_epochs=3,                  # Epoch sayısı
    per_device_train_batch_size=2,       # GPU başına batch boyutu
    per_device_eval_batch_size=4,       # GPU başına batch boyutu
    gradient_accumulation_steps=32,       # Gradient birikimi için adım sayısı
    bf16=True,                           # 16-bit floating-point
    fp16_opt_level="O2",                # Optimizasyon düzeyi (O1, O2, O3)
    dataloader_num_workers=4,
    #evaluation_strategy="no",           # Sadece eğitim (değerlendirme yapılmıyor)
    report_to="wandb",                    # WandB veya diğer araçlara raporlama yok
    save_total_limit=2,                  # Sadece son iki checkpoint'i sakla
    lr_scheduler_type="cosine",  # Cosine learning rate decay
    warmup_steps=1000,           # İlk 1000 adımda LR'yi yavaş yavaş artır
    weight_decay=0.01,           # AdamW kullanırken weight decay ekle
    optim="adamw_torch",         # Daha hızlı AdamW optimizasyonu
    callbacks=[wandb_callback]
    
)

# 5. Trainer Nesnesi ile Eğitimi Başlatın
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=final_dataset["train"],
    eval_dataset=final_dataset["validation"],
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics,
)

/home/hosman/anaconda3/envs/torchEnv/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_24815/3940627571.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train(resume_from_checkpoint="./SmolLM2-360M-Instruct-v1/checkpoint-31800")

There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
/home/hosman/anaconda3/envs/torchEnv/lib/python3.11/site-packages/transformers/trainer.py:3420: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any is

  0%|          | 0/94680 [00:00<?, ?it/s]/home/hosman/anaconda3/envs/torchEnv/lib/python3.11/site-packages/transformers/trainer.py:3083: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental f

{'eval_loss': 2.3989551067352295, 'eval_runtime': 6666.7541, 'eval_samples_per_second': 37.873, 'eval_steps_per_second': 9.468, 'epoch': 1.0}


 34%|███▎      | 31919/94680 [2:37:02<114:04:41,  6.54s/it]    

In [ ]:
# 6. Eğitilmiş Modeli Kaydedin
model.save_pretrained("./SmolLM2-360M-Instruct-v1")
tokenizer.save_pretrained("./SmolLM2-360M-Instruct-v1")

print("Eğitim tamamlandı ve model kaydedildi.")